In [ ]:
import sqlite3
from pymodbus.client import ModbusTcpClient
import time

# Define the Modbus TCP device's IP address and port

In [ ]:
device_ip = "127.0.0.1"  # Change to 127.0.0.1
device_port = 502        # Change to 502

# Create a connection to the Modbus TCP device

In [ ]:
client = ModbusTcpClient(device_ip, port=device_port)

# Define the Modbus slave unit ID (usually 1)

In [ ]:
unit_id = 1

# Define the starting register address and the number of registers to read

In [ ]:
start_address = 0
num_registers = 2

# Connect to the Modbus device

In [ ]:
client.connect()

# Connect to the SQLite database

In [ ]:
conn = sqlite3.connect("modbus_data.db")
cursor = conn.cursor()

# Create a table for the data if it doesn't exist

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS modbus_data (
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    holding_register_1 INT,
    holding_register_2 INT
)
""")

# Continuous data reading and logging

In [ ]:
while True:
    # Read holding registers
    response = client.read_holding_registers(start_address, num_registers, unit=unit_id)

    # Check if the request was successful
    if response.isError():
        print("Modbus Error:", response)
    else:
        data = response.registers
        #print("Holding Register Data:", data)

        # Insert the data into the database
        cursor.execute("INSERT INTO modbus_data (holding_register_1, holding_register_2) VALUES (?, ?)", (data[0], data[1]))

        # Commit the transaction
        conn.commit()

    # Wait for 5 seconds before reading data again
    time.sleep(5)

# Close the Modbus connection and the SQLite connection (this will never be reached in this example)

In [ ]:
client.close()
conn.close()